In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN2DTopoV1_ONOFF_local
from training import new_two_stage_training
from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

jax backend cpu


In [2]:
# Calculate A by using the default find_A function of SSN2DTopoV1_ONOFF_local when A is undefined
ssn_mid = SSN2DTopoV1_ONOFF_local(
    ssn_pars=ssn_pars, # got c_E and c_I as new parameters
    grid_pars=grid_pars,
    conn_pars=conn_pars_m,
    filter_pars=filter_pars,
    J_2x2 = ssn_layer_pars.J_2x2_m,
    gE = ssn_layer_pars.gE_m, 
    gI = ssn_layer_pars.gE_s
)

ssn_pars.A = ssn_mid.A
if ssn_pars.phases == 4:
    ssn_pars.A2 = ssn_mid.A2

####################### TRAINING PARAMETERS #############################
constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)

In [3]:
########### TRAINING ############
([ssn_layer_pars_dict, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    extra_stop=2,
)

New orientation map created for the middle layer.
#### NOT SAVING! ####
Training loss: 4.422994136810303 ¦ Validation -- loss: 4.040529727935791, true accuracy: 0.4399999976158142, at epoch 0, (time 18.347585916519165, 0.4893507957458496), 
Training accuracy: 0.3700000047683716, all losses[[2.4856381]
 [0.6306164]
 [0.6381918]
 [0.6685478]
 [0.       ]
 [4.422994 ]]
Training loss: 4.048635959625244 ¦ Validation -- loss: 4.242071151733398, true accuracy: 0.33000001311302185, at epoch 1, (time 17.479681968688965, 0.5081815719604492), 
Training loss: 2.253255844116211 ¦ Validation -- loss: 1.934602975845337, true accuracy: 0.8500000238418579, at epoch 20, (time 1.9721148014068604, 0.4299139976501465), 
Entering second stage at epoch 20
Training loss: 1.863214373588562 ¦ Validation -- loss: 2.1557059288024902, true accuracy: 0.7599999904632568, at epoch 1, (time 1.9721148014068604, 1.790384292602539)
Training accuracy: 0.8700000047683716, all losses[[3.0848053e-01]
 [6.3724339e-01]
 [6.378

In [4]:
from util import take_log
print(ssn_layer_pars_dict['logJ_2x2'][0]-take_log(ssn_layer_pars.J_2x2_m))
print(ssn_layer_pars_dict['logJ_2x2'][1]- take_log(ssn_layer_pars.J_2x2_s))

[[-0.05716056  0.05614817]
 [ 0.05472553 -0.05423924]]
[[-0.05114603  0.04433024]
 [ 0.04405046  0.0720316 ]]
